In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = os.path.join(os.getcwd(), '../data/train.csv')
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Our CRAZY machine learning thing :) ...

In [6]:
from preprocessing import *
from logistic_regression import *
import pandas as pd

In [ ]:
def cross_validation(tX, y, k=5, 
                     gamma=0.01, lambda_=10 
                     max_iter=5000, batch_size=128):
    k_folds = np.split(np.arange(y.shape[0]), axis=0, indeces_or_sections=k)
    accuracy = np.zeros(k)
    i = 0
    for k_fold in k_folds:
        train_mask = ~np.isin(np.arange(y.shape[0]), k_fold)
        w_init = np.random.randn(tX.shape[0], 1)
        
        x_train = tX[train_mask]
        y_train = y[train_mask]
        x_val = tX[~train_mask]
        y_val = y[~train_mask]
        
        w, loss = logistic_regression_reg(y=y_train, tx=x_train,
                                          initial_w=w_init,
                                          max_iter=max_iter, lambda_=lambda_,
                                          gamma=gamma, batch_size=batch_size)
        y_pred = logistic_pred(x_val, w)
        accuracy[i] = (y_pred==y_val).mean()
    accuracy_avg = accuracy.mean()
    return accuracy_avg
        

def hyperparamters_tunning(tX, y,
                           use_transformations, handling_outliers, degrees,
                           lrs, lambdas, epochs, batch_size, max_iter, k=5):
    best = {
        "transformation": None,
        "handling_outlier": None,
        "degree": None,
        "accuracy": 0,
        "lr": None,
        "lambda": None,
    }
    for use_transformation in use_transforamtions:
        for handling_outlier in handling_outliers:
            for degree in degrees:
                for lr in lrs:
                    for lambda_ in lambdas:
                        preprocessing = Preprocessing(
                            use_transformations=use_transfor,
                            handling_outliers=handling_outlier,
                            degrees=degree
                        )
                        
                        tX_preprocessed = preprocessing.preprocess(data_=tX)
                        accuracy = cross_validation(tX, y, k=k,
                                                    gamma=lr, lambda_=lambda_,
                                                    max_iter=max_iter, batch_size=batch_size)
                        if best['accuracy'] < accuracy:
                            print("Best is updated with accuracy:", accuracy)
                            best["transformation"] = use_transformation
                            best["handling_outlier"] = handling_outlier
                            best["degree"] = degree
                            best["accuracy"] = accuracy
                            best["lambda"] = lambda_
                            best["lr"] = lr
    return best

In [4]:
preprocessing = Preprocessing()
y[y==-1] = 0
y = y.astype(np.int)
preprocessing.replace_outliers_by_nan(tX)

#Delete rows from tX and y, which has more than 8 nan values in tX and 0 label in y
num_nan_in_row = np.isnan(tX).sum(axis=1)
mask = (y==0) & (num_nan_in_row>8)
y = y[~mask]
tX = tX[~mask]

In [ ]:
use_transformations = [True, False]
handling_outliers = ['fill_mean', 'remove']
lrs = [0.1, 0.01, 1]
lambdas = [1, 10, 100]
degree = np.arange(2, 5)

batch_size = 128

In [ ]:
best = hyperparamters_tunning(tX, y,
                              use_transformations=, handling_outliers, degrees,
                              lrs, lambdas, epochs, batch_size, max_iter, k=5)

In [ ]:
preprocessing = Preprocessing(use_transformations=use_transfor,
                              handling_outliers=handling_outlier,
                              degrees=degree)
                        
tX_preprocessed = preprocessing.preprocess(data_=tX)
w, loss = logistic_regression_reg(y=y_train, tx=x_train,
                                  initial_w=w_init,
                                  max_iter=max_iter, lambda_=lambda_,
                                  gamma=gamma, batch_size=batch_size)
y_pred = logistic_pred(tX, w)
(y_pred==y).mean()

## Generate predictions and save ouput in csv format for submission:

In [14]:
DATA_TEST_PATH = DATA_TRAIN_PATH = os.path.join(os.getcwd(), '../data/test.csv')
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [15]:
preprocessing = Preprocessing(
    use_transformations=use_transformation,
    handling_outliers = handling_outlier,
)
test_data, _ = preprocessing.preprocess(
    tX_test,
    train=False,
    models=models,
    degrees=degree
)

Preprocesing started!

Preprocessing ended



In [16]:
OUTPUT_PATH = 'prediction.csv' # TODO: fill in desired name of output file for submission
y_pred = model.predict(test_data)
res = y_pred>0.5
res = res.squeeze()
pred = -np.ones(res.shape)
pred[res] = 1
create_csv_submission(ids_test, pred, OUTPUT_PATH)